# LLM-Agent-Chem — Closed Loop Demo

In [1]:
from pathlib import Path
import random
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

from agent.llm import propose_smiles
from agent.retrieval import top_k_neighbors
from agent.scoring import score_knn, train_ridge, score_ridge

# Ensure results folders
Path('results/figures').mkdir(parents=True, exist_ok=True)

# Train ridge model (saves artifact if not present)
_ = train_ridge()
print('Ridge model is trained and ready.')


ModuleNotFoundError: No module named 'agent'

In [2]:
# Closed-loop optimization parameters
seed_smiles = 'CC'
n_iter = 15
delta = 1e-3  # minimum improvement to accept
p_explore = 0.1  # exploration probability

current = seed_smiles
current_score = score_ridge(current)
visited = set([current])

history = []
for t in tqdm(range(n_iter)):
    proposal, reason = propose_smiles(current)
    neigh = top_k_neighbors(proposal, k=5)
    knn_s = score_knn(proposal, neigh)
    ridge_s = score_ridge(proposal)

    accepted = False
    if ridge_s >= current_score + delta:
        accepted = True
    elif random.random() < p_explore and proposal not in visited:
        accepted = True

    if accepted:
        visited.add(proposal)
        current, current_score = proposal, ridge_s

    history.append({
        'iter': t,
        'original_smiles': current if accepted else current,
        'proposal': proposal,
        'reason': reason,
        'score_knn': float(knn_s),
        'score_ridge': float(ridge_s),
        'accepted': accepted,
        'current_score': float(current_score)
    })

df_hist = pd.DataFrame(history)
df_hist.head(), df_hist.tail()


NameError: name 'score_ridge' is not defined

In [ ]:
# Plot property vs iteration
plt.figure(figsize=(6, 4))
plt.plot(df_hist['iter'], df_hist['current_score'], marker='o')
plt.xlabel('Iteration')
plt.ylabel('Property (Ridge)')
plt.title('Closed-loop optimization')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('results/figures/loop.png', dpi=150)
plt.show()
